In [ ]:
!pwd

In [ ]:
!wget 'http://datakmeans.s3.amazonaws.com/uber-raw-data-apr14.csv'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# We are taking a portion of the data from the Uber data set
uber_data = pd.read_csv('/path/to/csv/uber-raw-data-apr14.csv', header=0)
print(uber_data.head())

In [ ]:
uber_data[:1100]

In [ ]:
import datetime


In [ ]:
#For Converting the Date column into seperate values 
date_col = []
for i in range(len(uber_data)):
    #print (i)
    for form in ("%m-%d-%Y  %H:%M", "%m/%d/%Y  %H:%M:%S"):
        try:
            #print (form)
            date_col.append(datetime.datetime.strptime(uber_data['Date_Time'][i], form))
        except ValueError:
            pass
            
    
#date_col = datetime.datetime.strptime(uber_data['Date_Time'], "%b-%d-%Y  %H:%M")

In [ ]:
year = []
month = []
day = []
hours = []
minu = []
for i in range(len(date_col)):
    year.append(date_col[i].year)
    month.append(date_col[i].month)
    day.append(date_col[i].day)
    hours.append(date_col[i].hour)
    minu.append(date_col[i].minute)

In [ ]:
uber_data1 = uber_data
uber_data1['year'] = year

In [ ]:
uber_data1['month'] = month
uber_data1['day'] = day
uber_data1['hour'] = hours
uber_data1['minute'] = minu

In [ ]:
uber_data1[:10]

In [ ]:
type(uber_data['day'])

In [ ]:
#uber_final = uber_data1[['Lat','Lon','Base','year','month','day','hour','minute']]
uber_loc = uber_data1[['Lat','Lon']]

In [ ]:
uber_loc[1:10]

In [ ]:
uber_final = uber_loc.as_matrix().astype(np.float32)

In [ ]:
from sagemaker import KMeans
from sagemaker import get_execution_role

role = get_execution_role()
print(role)

# TODO: Edit the bucket and data_location
bucket = "sagemaker-sai1"
data_location = "sagemaker-sai1"


data_location = 's3://{}/data/location'.format(bucket)
output_location = 's3://{}/output/location'.format(bucket)

print('training data will be uploaded to: {}'.format(data_location))
print('training artifacts will be uploaded to: {}'.format(output_location))

kmeans = KMeans(role=role,
                train_instance_count=1,
                train_instance_type='ml.p2.xlarge',
                output_path=output_location,
                k=5,
                data_location=data_location)

In [ ]:
# Run the model using the Numpy array
# hint - Use method record_set(your numpy array) in kmeans package
%%time

# TODO: Edit the argument to 'fit'
kmeans.fit(kmeans.??(uber_final))

In [ ]:
# Deploy the model we just trained behind a real-time hosted endpoint using kmeans.deploy()
# Specify the initial_instance_count and instance_type as arguments in deploy()
%%time

kmeans_predictor = kmeans.deploy(initial_instance_count=1,
                                 instance_type='ml.p2.xlarge')

In [ ]:
# To Validate the model for use, Take a small subset of the data and 
# see the results of the clustering using kmeans_predictor.predict(your_array[Specify a range])

# TODO: Edit the predictor function to call
test_result = kmeans_predictor.??(uber_final[1:100])

In [ ]:
%%time

#result = kmeans_predictor.predict(s)
clusters = [r.label['closest_cluster'].float32_tensor.values[0] for r in test_result]
i = 0
 
    
for r in test_result:
    out = {
        "Date"     : uber_data1['Date_Time'].iloc[i],
        "Latitude":uber_data1['Lat'].iloc[i],
        "Longitude" : uber_data1['Lon'].iloc[i],  
        "closest_cluster" :  r.label['closest_cluster'].float32_tensor.values[0],
        "Base Code" : uber_data1['Base'].iloc[i],  
    }    
    print(out) 
    i = i + 1

In [ ]:
## TODO

#So, Next we may use the clusters for finding,
 #1. Number of pickups in the year 2014 with closest cluster = 1 on month 4 and day 1?
 #2. How many pickups occurred in each cluster? 
 #3. No of cabs booked in a particular clustering, day, month, hour.